In [1]:
import os
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from dataset_tools import *

In [2]:
def generate_ties(dataset,out_dir,length,tie_size,min_r=0,min_roi = 0.5):
    for in_dir,out_dir in iterate_folders(dataset,out_dir):
        iter = 0
        motion_ties = None
        for batch_num,(imgs,masks) in enumerate(iterate_bathced(in_dir,length)):
            print in_dir+' %d %d\r'%(batch_num,iter),
            h,w = tie_size,tie_size
            cols = imgs.shape[2]//w
            rows = imgs.shape[1]//h
            imgs = imgs[:,:h*rows,:w*cols,:]
            masks = masks[:,:h*rows,:w*cols]
            imgs = np.transpose(imgs.reshape((-1,rows,h,cols,w,3)),(1,3,0,2,4,5)).reshape(cols*rows,len(imgs),h,w,3)
            masks = np.transpose(masks.reshape((-1,rows,h,cols,w)),(1,3,0,2,4)).reshape(cols*rows,len(masks),h,w)
            
            masks_mean_motion = np.zeros_like(masks)
            masks_mean_motion[masks > 240] = 1
            masks_mean_motion = masks_mean_motion.mean((2,3))
            
            
            for i in range(len(masks)):
                m_r = masks_mean_motion[i].mean() 
                m_roi = masks_mean_roi[i].mean()
                if(m_r >= min_r and m_roi >= min_roi):
                    im = ties_to_image(imgs[i])
                    mask = ties_to_image(masks[i])
                    cv2.imwrite(out_dir+'/%d_input.jpg'%(iter),im)
                    cv2.imwrite(out_dir+'/%d_mask.png'%(iter),mask)
                    iter=iter+1

            m_ties = imgs.reshape((-1,h,w,3))[masks_mean_motion.flatten()>=.97]
            if(len(m_ties) > 0):
                if not(motion_ties is None):
                    motion_ties = np.concatenate((motion_ties,m_ties))
                else:
                    motion_ties = m_ties
        if not (motion_ties is None):
            for i in range(int(np.ceil(float(len(motion_ties))/50.))):
                im = ties_to_image(motion_ties[i*50:min((i+1)*50,len(motion_ties))],5,10)
                cv2.imwrite(out_dir+'/%d_motion.jpg'%(i),im)
            print in_dir+' %d ties=%d motion=%d'%(batch_num,iter,len(motion_ties))
        else:
            print in_dir+' %d ties=%d motion=%d'%(batch_num,iter,0)


In [3]:
generate_ties('../gmm_segmentation/dataset','train_ties1',256,32,-1.,0.95)

../gmm_segmentation/dataset/badWeather/skating 11 ties=1049 motion=5138
../gmm_segmentation/dataset/badWeather/snowFall 21 ties=3616 motion=4204
../gmm_segmentation/dataset/badWeather/wetSnow 21 ties=0 motion=0
../gmm_segmentation/dataset/baseline/office 4 ties=318 motion=115
../gmm_segmentation/dataset/baseline/pedestrians 2 ties=227 motion=0
../gmm_segmentation/dataset/baseline/PETS2006 2 ties=1156 motion=432
../gmm_segmentation/dataset/cameraJitter/boulevard 2 ties=0 motion=0
../gmm_segmentation/dataset/cameraJitter/sidewalk 2 ties=0 motion=0
../gmm_segmentation/dataset/cameraJitter/traffic 1 ties=118 motion=812
../gmm_segmentation/dataset/dynamicBackground/canoe 0 ties=51 motion=0
../gmm_segmentation/dataset/dynamicBackground/fall 10 ties=3627 motion=11867
../gmm_segmentation/dataset/dynamicBackground/fountain01 2 ties=326 motion=0
../gmm_segmentation/dataset/dynamicBackground/fountain02 2 ties=351 motion=0
../gmm_segmentation/dataset/dynamicBackground/overpass 6 ties=434 motion=29

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/nvvaulin/.local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/nvvaulin/.local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/nvvaulin/.local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python2.7/inspect.py", line 1049, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/lib/python2.7/inspect.py", line 1009, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/usr/lib/python2.7/inspect.py", line 500, in getmodule


IndexError: string index out of range

In [15]:
generate_ties('../gmm_segmentation/test_dataset','test_ties',256,32,0.01,0.95)

../gmm_segmentation/test_dataset/badWeather/blizzard 26 ties=591 motion=3927
../gmm_segmentation/test_dataset/baseline/highway 5 ties=64 motion=559
../gmm_segmentation/test_dataset/cameraJitter/badminton 3 ties=0 motion=220
../gmm_segmentation/test_dataset/dynamicBackground/boats 30 ties=6 motion=0
../gmm_segmentation/test_dataset/intermittentObjectMotion/abandonedBox 16 ties=1 motion=0
../gmm_segmentation/test_dataset/lowFramerate/port_0_17fps 10 ties=2 motion=0
../gmm_segmentation/test_dataset/nightVideos/bridgeEntry 8 ties=11 motion=0
../gmm_segmentation/test_dataset/PTZ/continuousPan 5 ties=171 motion=759
../gmm_segmentation/test_dataset/shadow/backdoor 6 ties=78 motion=37
../gmm_segmentation/test_dataset/thermal/corridor 20 ties=246 motion=3143
../gmm_segmentation/test_dataset/turbulence/turbulence0 18 ties=103 motion=0


In [5]:
import shutil

In [7]:
def purge(dir):
    for i in os.listdir(dir):
        for j in os.listdir(dir+'/'+i):
            p = dir+'/'+i+'/'+j
            l = [k[:-4] for k in os.listdir(p)]
            if( not('0_motion' in l)):
                print(p,'no motion ',len(l))
                shutil.rmtree(p)
            else:
                print(p,len(l))
purge('train_ties1')          

('train_ties1/badWeather/skating', 2201)
('train_ties1/badWeather/snowFall', 7317)
('train_ties1/baseline/office', 639)
('train_ties1/baseline/PETS2006', 2321)
('train_ties1/cameraJitter/traffic', 253)
('train_ties1/dynamicBackground/fall', 7492)
('train_ties1/dynamicBackground/overpass', 874)
('train_ties1/intermittentObjectMotion/sofa', 965)
('train_ties1/lowFramerate/turnpike_0_5fps', 98)
('train_ties1/nightVideos/streetCornerAtNight', 1924)
('train_ties1/PTZ/intermittentPan', 750)
('train_ties1/shadow/busStation', 363)
('train_ties1/shadow/copyMachine', 7402)
('train_ties1/shadow/cubicle', 3520)
('train_ties1/shadow/peopleInShade', 427)
('train_ties1/thermal/diningRoom', 1441)
('train_ties1/thermal/lakeSide', 2619)
('train_ties1/thermal/library', 2100)
('train_ties1/thermal/park', 199)
('train_ties1/turbulence/turbulence1', 3247)
